## Imports

In [2]:
cd ../pathofexile/

/Users/steve/Dropbox/Programming/Path of Exile project/pathofexile


In [3]:
import pathofexile.api as poe
import pathofexile.ladder as ladder
import sqlite3

In [4]:
cd ../src

/Users/steve/Dropbox/Programming/Path of Exile project/src


In [5]:
import poe_utils

reload(poe_utils)

<module 'poe_utils' from 'poe_utils.pyc'>

First test that we can connect to the Path of Exile API:

In [9]:
test_leagues = poe.get_leagues(league_type='season', season='Season10', compact_info=0, league_limit=1, league_offset=0)
print test_leagues[0]

{u'startAt': u'2015-01-02T16:00:00Z', u'description': u'This is the Season Ten Signature Event.\n\nPlease check the Event Forums for more details and points awarded.', u'registerAt': u'2015-01-02T15:30:00Z', u'url': u'http://pathofexile.com/forum/view-thread/1156124', u'endAt': u'2015-01-02T17:30:00Z', u'id': u'90m Fixed Seed Brutal (S10S049)', u'rules': [{u'description': u'A character killed in Hardcore is moved to its parent league.', u'id': 4, u'name': u'Hardcore'}, {u'description': u'You may not party in this league.', u'id': 24, u'name': u'Solo'}], u'event': True}


# Connect to an sqlite table

In [18]:
conn = sqlite3.connect('../data/poe_races.db')
c = conn.cursor()

## Scrape leagues

In [ ]:
poe_utils.all_leagues_to_table(c, conn, 'leagues')
#conn.commit() # To actually push changes to the table

Now we can check that we have leagues in our table and they parsed correctly.

In [36]:
c.execute('''
          SELECT * FROM leagues
          Limit 2;
          ''')
c.fetchall()

[(u'Standard',
  u'The default game mode.',
  u'2013-01-23T21:00:00Z',
  u'None',
  u'None',
  u'http://pathofexile.com/forum/view-thread/71278',
  u'[]',
  0),
 (u'Hardcore',
  u'A character killed in the Hardcore league is moved to the Standard league.',
  u'2013-01-23T21:00:00Z',
  u'None',
  u'None',
  u'http://pathofexile.com/forum/view-thread/71276',
  u"[{u'description': u'A character killed in Hardcore is moved to its parent league.', u'id': 4, u'name': u'Hardcore'}]",
  0)]

In [34]:
c.execute('''
          SELECT * FROM leagues
          WHERE rowid > 100
          LIMIT 2;
          ''')
c.fetchall()

[(u'Jan 5 Party HC Race',
  u'A 3-hour Hardcore race event with partying enabled',
  u'2013-01-04T19:00:00Z',
  u'2013-01-04T22:00:00Z',
  u'2013-01-04T18:30:00Z',
  u'http://pathofexile.com/forum/view-thread/64074',
  u"[{u'description': u'A character killed in Hardcore is moved to its parent league.', u'id': 4, u'name': u'Hardcore'}]",
  1),
 (u'Jan 6 Solo HC Race',
  u'A 3-hour Solo Hardcore race event',
  u'2013-01-05T21:00:00Z',
  u'2013-01-06T00:00:00Z',
  u'2013-01-05T20:30:00Z',
  u'http://pathofexile.com/forum/view-thread/64075',
  u"[{u'description': u'A character killed in Hardcore is moved to its parent league.', u'id': 4, u'name': u'Hardcore'}, {u'description': u'You may not party in this league.', u'id': 24, u'name': u'Solo'}]",
  1)]

In [37]:
c.execute('SELECT COUNT(*) FROM leagues;')
c.fetchall()

[(5222,)]

## Scrape ladders

In [21]:
league_ids = poe_utils.get_leagues_from_table(c,'leagues')
poe_utils.all_ladders_to_table(c, conn, league_ids, 'ladders', start=0)
conn.commit()

Check that the ladder got added to the table correctly.

In [38]:
league_ids = poe_utils.get_leagues_from_table(c,'leagues')
ladder1 = ladder.retrieve(league_ids[0])
print ladder1[0]

{u'account': {u'challenges': {u'total': 18}, u'twitch': {u'name': u'helmannn'}, u'name': u'Helman'}, u'character': {u'level': 22, u'class': u'Duelist', u'experience': 1501660, u'name': u'Twitch_Tv___Helmannn'}, u'rank': 1, u'dead': False, u'online': False}


## Deduplicate ladder entries

In [25]:
# This checks the number of duplicates
c.execute('''
          SELECT * FROM ladders
          WHERE rowid NOT IN(
             SELECT  MIN(rowid)
             FROM    ladders
             GROUP BY id, account, twitch, challenges, character, rank, class, experience, dead
             );
          ''')
len(c.fetchall())

324884

I had some some duplicate records in the ladders table (324884 to be exact). I removed them using the code below (don't forget to commit!).

In [26]:
c.execute('''
          DELETE FROM ladders
          WHERE rowid NOT IN(
             SELECT  MIN(rowid)
             FROM    ladders
             GROUP BY id, account, twitch, challenges, character, rank, class, experience, dead
             );
          ''')

In [28]:
conn.commit()

In [39]:
conn.close()